Импорты

In [ ]:
!pip install langdetect

In [4]:
import pandas as pd
from langdetect import detect

Соединеним файлы в  единый датасет. Для понимания на каком языке написан датасет, используем библиотеку langdetect

In [28]:
path = 'data/'

langs = []
langCount = {}
train_table = {'article': [], 'abstract': []}
ind = {}
for j in range(5):
    table = pd.read_parquet(path + f'train-0000{j}-of-00005.parquet')
    for i in table.values:
        if i[0] != '':
            train_table['article'].append(i[0])
            train_table['abstract'].append(i[1])

            lang = detect(i[1])
            if lang not in langs:
                langs.append(lang)
                langCount[lang]=1
                ind[lang] = i[1]
            else:
                langCount[lang]+=1
    print('iter:', j+1)


train = pd.DataFrame(train_table)
train

iter: 1
iter: 2
iter: 3
iter: 4
iter: 5


,article,abstract
0,a recent systematic analysis showed that in 20...,background : the present study was carried out...
1,it occurs in more than 50% of patients and may...,backgroundanemia in patients with cancer who a...
2,"tardive dystonia ( td ) , a rarer side effect ...",tardive dystonia ( td ) is a serious side effe...
3,"lepidoptera include agricultural pests that , ...",many lepidopteran insects are agricultural pes...
4,syncope is caused by transient diffuse cerebra...,we present an unusual case of recurrent cough ...
...,...,...
117103,eukaryotic cells depend on vesicle - mediated ...,long - distance trafficking of membranous stru...
117104,as regards the selection criteria of the post ...,aims and objectives : to study the stress conc...
117105,in most of the peer review publications in the...,abstractbackgroundthe objective of this study ...
117106,the reveal registry is a longitudinal registry...,background : patients with pulmonary arterial ...


Теперь можно узнать на каких языках написаны статьи, а также сколько статей написаны на определённом языке

In [30]:
langs, langCount

(['en', 'fr', 'it'], {'en': 117106, 'fr': 1, 'it': 1})

Почти все статьи написаны на английском. Выкинем лишние, чтобы улучшить качество обучения модели.

In [34]:
for i in langs:
    if i == 'en': continue
    train = train[train['abstract']!= ind[i]]
    
train.shape

(117106, 2)

In [71]:
#проверка на наличие заглавных букв
train['article'].apply(lambda x: x.istitle()).sum()

0

В ходе анализа данных мы поняли, что в датасете находятся статьи, связанные с медицинскими иследованиями, биологией, химией.
Главная особенность научных статей -  большое количество цифр, сокращений, вставных конструкций. Так, например в строке 65581 есть проценты, нумерование пунктов. В нулевой строке же в виде вставных конструкций фигурирует вероятность, которая обозначается буквой p. То есть модель должна научиться понимать сокращения и формулы. Проблемой так же может быть возникновение узконаправленных и редких терминов. В датасете так же было большое количество строк, на вход которых подавалась пустая строка, а на выходе был сокращенный текст. Данную проблему мы решили на момент загрузки датасета

В датасете в столбцах article и abstract нет заглавных букв, а так же все знаки препинания отделены пробелами, то есть датасет уже токенизирован. Это значит, что нам не нужно обрабатывать данные. При внедрении модели в продукт надо будет подавать ей токенизированный текст.

$$SOSI\;Artem\;Gonharov$$

0